### Dataset Generation

In [2]:
from shapely.geometry import Point, Polygon
from matplotlib import pyplot as plt
from typing import List

import random

class polygon:
    def __init__(self, n):
        self.poly  = Polygon()
        self.sides = n
        self.range = 50

        if(n*10 > self.range):
            self.range = n*10
    
    def generate(self):
        random.SystemRandom()
        x = random.sample(range(-self.range, self.range), self.sides)
        y = random.sample(range(-self.range, self.range), self.sides)
        z = list(zip(x,y))
        self.poly = Polygon(z)
        self.poly = self.poly.convex_hull
        n = len(self.poly.exterior.xy[0])-1
        while n <self.sides:
            random.SystemRandom()
            x, y   = (random.randint(-self.range, self.range),
                      random.randint(-self.range, self.range))

            x1, y1 = self.poly.exterior.xy

            x1.append(x)
            y1.append(y)
            z = list(zip(x1, y1))
            self.poly = Polygon(z)
            self.poly = self.poly.convex_hull
            n = len(self.poly.exterior.xy[0])-1
        
    def plot(self):
        x, y = self.poly.exterior.xy
        plt.plot(x, y)
        plt.show()

    def getsides(self):
        return self.sides

### Minimum Cost Triangulation - Brute Force Approach

In [7]:
import math
def cost(P:Polygon, i, j, k):
    p1 = Point(P.poly.exterior.coords[i])
    p2 = Point(P.poly.exterior.coords[j])
    p3 = Point(P.poly.exterior.coords[k])

    dist = p1.distance(p2) + p2.distance(p3) + p3.distance(p1)
    return dist

    
def min_cost_brute(P:Polygon, i, j):  
    
    if(j < i+2):
        return 0

    res = 9999999999
    for k in range (i+1, j):
        minimum = min_cost_brute(P, i, k) + min_cost_brute(P, k, j) + cost(P, i, k, j)
        if minimum <= res:
            res = minimum

    return round(res,4)


In [29]:
### automated dataset
cost_arr_brute = []
for i in range(3,15):
    p = polygon(i)
    p.generate()
    cost_arr_brute.append(min_cost_brute(p,0, i-1))

In [30]:
print(cost_arr_brute)

[146.3268, 390.4312, 541.1209, 929.6498, 1099.183, 1430.8171, 1909.0197, 2400.0254, 2521.4234, 3486.5557, 3849.6731, 4159.1162]


### Minimum Cost Triangulation - Dynamic Programming

In [22]:
import math
import sys
def dist(x, y):
    return math.sqrt((x[0] - y[0]) * (x[0] - y[0]) + (x[1] - y[1]) * (x[1] - y[1]))

def min_cost_dynamic(P:Polygon, i, j):
    n = P.getsides()
    vertices = list(P.poly.exterior.coords)
    T = [[0.0] * n for _ in range(n)]
 
    for diagonal in range(n):
        i = 0
        for j in range(diagonal, n):
            if j >= i + 2:
                T[i][j] = sys.maxsize
                for k in range(i + 1, j):
                    weight = dist(vertices[i], vertices[j]) + \
                            dist(vertices[j], vertices[k]) + \
                            dist(vertices[k], vertices[i])

                    T[i][j] = min(T[i][j], weight + T[i][k] + T[k][j])
            i += 1
 
    return T[0][-1]


In [27]:
cost_arr_dynamic = []
for i in range(3,15):
    p = polygon(i)
    p.generate()
    arr = list(p.poly.exterior.coords)
    cost_arr_dynamic.append(min_cost_brute(p,0, i-1))

In [28]:
print(cost_arr_dynamic)

[141.6375, 225.4847, 497.5003, 824.5405, 1152.2218, 1299.552, 1869.3582, 2660.9246, 2544.7558, 3388.1992, 3936.7042, 4316.9714]
